In [1]:
import os 
import sys
sys.path.append('/home/gaokun/kk/')
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import precision_score, recall_score, accuracy_score, balanced_accuracy_score
print(sys.path)
from aix360_kun.aix360.algorithms.rule_induction.ripper import RipperExplainer
import time

['/home/gaokun/anaconda3/envs/tf24/lib/python311.zip', '/home/gaokun/anaconda3/envs/tf24/lib/python3.11', '/home/gaokun/anaconda3/envs/tf24/lib/python3.11/lib-dynload', '', '/home/gaokun/anaconda3/envs/tf24/lib/python3.11/site-packages', '/home/gaokun/kk/']


/home/gaokun/anaconda3/envs/tf24/lib/python3.11/site-packages/numba/np/ufunc/parallel.py:371: NumbaWarning: The TBB threading layer requires TBB version 2021 update 6 or later i.e., TBB_INTERFACE_VERSION >= 12060. Found TBB_INTERFACE_VERSION = 12050. The TBB threading layer is disabled.
  warnings.warn(problem)


Importing dev version v0.982 of RIPPER
Importing dev version v0.982 of RIPPER


# Rule Induction using RIPPER

## Binary classification with a random 20% test set

We read the adult dataset from the UCI repository. The goal is to learn a rule describing people who earn more than 50K.

In [3]:
data_type = {'age': float,
             'workclass': str,
             'fnlwgt': float,
             'education': str,
             'education-num': float,
             'marital-status': str,
             'occupation': str,
             'relationship': str,
             'race': str,
             'sex': str,
             'capital-gain': float,
             'capital-loss': float,
             'native-country': str,
             'hours-per-week': float,
             'label': str}

col_names = ['age', 'workclass', 'fnlwgt', 'education',
             'education-num', 'marital-status', 'occupation',
             'relationship', 'race', 'sex',
             'capital-gain', 'capital-loss', 'hours-per-week',
             'native-country', 'label']

df = pd.read_csv('https://archive.ics.uci.edu/ml/machine-learning-databases/adult/adult.data',
                 header=None,
                 delimiter=', ',
                 engine='python',
                 names=col_names,
                 dtype=data_type)

### Comlum names shall not contain whitespace or arithmetic operators (+, -, *, /)
We eventually output the rule set in TRXF format, where compound features are supported by parsing an expression string. So simple features like column names of a data frame must not contain these so that they are parsed as a single variable rather than an expression.

In [4]:
df.columns = df.columns.str.replace('-', '_')
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 32561 entries, 0 to 32560
Data columns (total 15 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   age             32561 non-null  float64
 1   workclass       32561 non-null  object 
 2   fnlwgt          32561 non-null  float64
 3   education       32561 non-null  object 
 4   education_num   32561 non-null  float64
 5   marital_status  32561 non-null  object 
 6   occupation      32561 non-null  object 
 7   relationship    32561 non-null  object 
 8   race            32561 non-null  object 
 9   sex             32561 non-null  object 
 10  capital_gain    32561 non-null  float64
 11  capital_loss    32561 non-null  float64
 12  hours_per_week  32561 non-null  float64
 13  native_country  32561 non-null  object 
 14  label           32561 non-null  object 
dtypes: float64(6), object(9)
memory usage: 3.7+ MB


In [5]:
TARGET_COLUMN = 'label'
print(df.head())

    age         workclass    fnlwgt  education  education_num  \
0  39.0         State-gov   77516.0  Bachelors           13.0   
1  50.0  Self-emp-not-inc   83311.0  Bachelors           13.0   
2  38.0           Private  215646.0    HS-grad            9.0   
3  53.0           Private  234721.0       11th            7.0   
4  28.0           Private  338409.0  Bachelors           13.0   

       marital_status         occupation   relationship   race     sex  \
0       Never-married       Adm-clerical  Not-in-family  White    Male   
1  Married-civ-spouse    Exec-managerial        Husband  White    Male   
2            Divorced  Handlers-cleaners  Not-in-family  White    Male   
3  Married-civ-spouse  Handlers-cleaners        Husband  Black    Male   
4  Married-civ-spouse     Prof-specialty           Wife  Black  Female   

   capital_gain  capital_loss  hours_per_week native_country  label  
0        2174.0           0.0            40.0  United-States  <=50K  
1           0.0         

### The rule induction trains for specific 'foreground' aka 'positive' value of the target label, which we set to '>50K' below. This means that the rule set will characterize the set of adults who earn more than 50K).

In [6]:
POS_VALUE = '>50K' # Setting positive value of the label for which we train
values_dist = df[TARGET_COLUMN].value_counts()
print('Positive value {} occurs {} times.'.format(POS_VALUE,values_dist[POS_VALUE]))
print(values_dist)
# This is distribution of the two values of the target label

Positive value >50K occurs 7841 times.
label
<=50K    24720
>50K      7841
Name: count, dtype: int64


In [8]:
train, test = train_test_split(df, test_size=0.2, random_state=42)
# Split the data set into 80% training and 20% test set
print('Training set:')
print(train[TARGET_COLUMN].value_counts())
print('Test set:')
print(test[TARGET_COLUMN].value_counts())

y_train = train[TARGET_COLUMN]
x_train = train.drop(columns=[TARGET_COLUMN])

y_test = test[TARGET_COLUMN]
x_test = test.drop(columns=[TARGET_COLUMN])
# Split data frames into features and label
display(x_train)
display(y_train)

Training set:
label
<=50K    19778
>50K      6270
Name: count, dtype: int64
Test set:
label
<=50K    4942
>50K     1571
Name: count, dtype: int64


,age,workclass,fnlwgt,education,education_num,marital_status,occupation,relationship,race,sex,capital_gain,capital_loss,hours_per_week,native_country
5514,33.0,Local-gov,198183.0,Bachelors,13.0,Never-married,Prof-specialty,Not-in-family,White,Female,0.0,0.0,50.0,United-States
19777,36.0,Private,86459.0,Assoc-voc,11.0,Married-civ-spouse,Exec-managerial,Husband,White,Male,0.0,1887.0,50.0,United-States
10781,58.0,Self-emp-not-inc,203039.0,9th,5.0,Separated,Craft-repair,Not-in-family,White,Male,0.0,0.0,40.0,United-States
32240,21.0,Private,180190.0,Assoc-voc,11.0,Married-civ-spouse,Farming-fishing,Husband,White,Male,0.0,0.0,46.0,United-States
9876,27.0,Private,279872.0,Some-college,10.0,Divorced,Other-service,Not-in-family,White,Male,0.0,0.0,40.0,United-States
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
29802,47.0,Private,359461.0,Bachelors,13.0,Married-civ-spouse,Craft-repair,Husband,White,Male,0.0,0.0,40.0,United-States
5390,31.0,Private,147215.0,12th,8.0,Divorced,Other-service,Unmarried,White,Female,0.0,0.0,21.0,United-States
860,18.0,Private,216284.0,11th,7.0,Never-married,Adm-clerical,Own-child,White,Female,0.0,0.0,20.0,United-States
15795,50.0,Self-emp-not-inc,54261.0,HS-grad,9.0,Married-civ-spouse,Farming-fishing,Husband,White,Male,0.0,0.0,84.0,United-States


5514      >50K
19777     >50K
10781    <=50K
32240    <=50K
9876     <=50K
         ...  
29802    <=50K
5390     <=50K
860      <=50K
15795    <=50K
23654    <=50K
Name: label, Length: 26048, dtype: object

### Instantiate the Ripper estimator and train it using default parameters

In [9]:
estimator = RipperExplainer()

start_time = time.time()
estimator.fit(x_train, y_train, target_label=POS_VALUE) # Run RIPPER rule induction
end_time = time.time()
print('Training time (sec): ' + str(end_time - start_time))

# compute performance metrics on test set
y_pred = estimator.predict(x_test)

print('Accuracy:', accuracy_score(y_test, y_pred))
print('Balanced accuracy:', balanced_accuracy_score(y_test, y_pred))
print('Precision:', precision_score(y_test, y_pred, pos_label=POS_VALUE))
print('Recall:', recall_score(y_test, y_pred, pos_label=POS_VALUE))

RIPPER optimization process: 100%|██████████| 2/2 [00:08<00:00,  4.46s/it]

Training time (sec): 11.570802688598633
Accuracy: 0.8441578381698143
Balanced accuracy: 0.7392636183805987
Precision: 0.7460176991150442
Recall: 0.5366008911521324


### Extract the rule set

In [10]:
trxf_ruleset = estimator.explain()
print(str(trxf_ruleset))

if
([marital_status == Married-civ-spouse] ^ [education_num >= 9.0] ^ [age >= 37.0] ^ [education_num <= 11.0] ^ [hours_per_week >= 38.0] ^ [workclass == Federal-gov]) v
([marital_status == Married-civ-spouse] ^ [education_num >= 14.0] ^ [capital_loss >= 1741.0]) v
([marital_status == Married-civ-spouse] ^ [education_num >= 9.0] ^ [education_num <= 11.0] ^ [hours_per_week >= 38.0] ^ [fnlwgt >= 139000.0] ^ [fnlwgt <= 145098.0] ^ [age <= 54.0] ^ [age >= 51.0]) v
([marital_status == Married-civ-spouse] ^ [education_num >= 9.0] ^ [education_num <= 10.0] ^ [hours_per_week >= 38.0] ^ [age <= 59.0] ^ [hours_per_week <= 46.0] ^ [age >= 58.0] ^ [occupation == Adm-clerical]) v
([marital_status == Married-civ-spouse] ^ [education_num >= 9.0] ^ [education_num <= 11.0] ^ [fnlwgt >= 140664.0] ^ [hours_per_week >= 44.0] ^ [hours_per_week <= 52.0] ^ [age >= 57.0] ^ [fnlwgt <= 155256.0]) v
([marital_status == Married-civ-spouse] ^ [education_num >= 9.0] ^ [education_num <= 10.0] ^ [hours_per_week >= 40.

## Export the resulting ruleset to a PMML file
### Construct a RuleSetClassifier object
A rule set by itself is merely a description of the given concept/target. Therefore, to use rule sets for a binary classification task, we must specify how to deal with potential overlaps between rule sets. For example, we could have learned 2 rule sets: one for >50K and another for <=50K. For instances where both rule sets are triggered, how do we classify that instance? There are 3 rule selection methods supported in PMML: First Hit, Weighted Sum, and Weighted Max. See here for more info: https://dmg.org/pmml/v4-4/RuleSet.html#xsdElement_RuleSelectionMethod. If we only learn a rule set for a single label, we can set a default label to which instances will be classified when the learned rule set does not trigger. 

In our case, since we only learn a rule set for a single label and use the default label for the rest, all 3 rule selection methods will have the same effect. However, if a rule selection method other than FirstHit is chosen, we need to compute the weights and confidence values for each rule.

In [6]:
import aix360.algorithms.rule_induction.trxf.classifier.ruleset_classifier as trxf_classifier
import aix360.algorithms.rule_induction.trxf.pmml_export as pmml
classifier = trxf_classifier.RuleSetClassifier([trxf_ruleset],
                                               rule_selection_method=trxf_classifier.RuleSelectionMethod.WEIGHTED_MAX,
                                               confidence_metric=trxf_classifier.ConfidenceMetric.LAPLACE,
                                               weight_metric=trxf_classifier.WeightMetric.CONFIDENCE,
                                               default_label='1.0')
classifier.update_rules_with_metrics(x_test, y_test)

NameError: name 'trxf_ruleset' is not defined

### Export the TRXF classifier to a PMML document

In [10]:
reader = pmml.TrxfReader()
reader.load_data_dictionary(x_test)
serializer = pmml.NyokaSerializer()
exporter = pmml.PmmlExporter(reader, serializer)
with open("adult_weighted_max.pmml", "w") as text_file:
    text_file.write(exporter.export(classifier))